### Libraries

In [20]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your       (x^1)
     [0.55, 0.87, 0.66], # journey    (x^2)
     [0.57, 0.85, 0.64], # starts     (x^3)
     [0.22, 0.58, 0.33], # with       (x^4)
     [0.77, 0.25, 0.10], # one        (x^5)
     [0.05, 0.80, 0.55], # step       (x^6)
    ]
)

In [21]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Corresponding words
words = ['Your','journey','starts','with','one','step']

# Extract x, y, z coordinates
x_coords = inputs[:, 0].numpy()
y_coords = inputs[:, 1].numpy()
z_coords = inputs[:, 2].numpy()

In [22]:
# Notes: This code picks one word (query) and calculates how similar every other word is to it using dot products, storing the results as attention scores.
query = inputs[1]   # 2nd input token is the query i.e. journey

attn_scores_2 = torch.empty(inputs.shape[0])   # initialize it as an empty tensor with that number of words
for i, x_i in enumerate(inputs):    # Loop through all the inputs
    attn_scores_2[i] = torch.dot(x_i, query)  # compute the dot product (transpose not nee)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


### Normalization

In [23]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


### Naive Softmax

In [24]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0) # We are taking all the inputs in a row

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


### PyTorch implementation of Softmax

In [25]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


### Context Vector

In [26]:
query = inputs[1]  # 2nd input token is the query (journey)

context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i  # Scaling each input vector according to its attention weight

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### Attention scores (inputs)<br>
#### For loops

In [27]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)
# For loops are generally slow and we can achieve the same results by using matrix computation

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


#### Matrix multiplication - linear algebra (computationally fast)

In [28]:
attn_scores = inputs @ inputs.T
# print(inputs)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


#### Normalization using PyTorch softmax

In [29]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [30]:
# Example for row 2
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum", row_2_sum)
print("All row sums", attn_weights.sum(dim=-1))

Row 2 sum 1.0
All row sums tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


#### Context Vector

In [31]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
